In [29]:
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate
from datetime import timedelta

In [44]:
@tool
def interactive_queue( node_number: int, node_type: str, time:int, project:str) -> int:
    """Interactive queue on number of node for given time in hours in what project"""
    formatted_time = timedelta(hours=time)
    return print(f"qsub -I -l select={node_number} -l filesystems=home:eagle -l walltime={formatted_time} -q {node_type} -A {project}")

In [45]:
interactive_queue.invoke("5 node debug, 1 hour, HPCBot Project")

ValidationError: 4 validation errors for interactive_queue
node_number
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='5 node debug, 1 hour, HPCBot Project', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/int_parsing
node_type
  Field required [type=missing, input_value={'node_number': '5 node d...1 hour, HPCBot Project'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
time
  Field required [type=missing, input_value={'node_number': '5 node d...1 hour, HPCBot Project'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing
project
  Field required [type=missing, input_value={'node_number': '5 node d...1 hour, HPCBot Project'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

In [38]:
### LLM
from langchain_ollama import ChatOllama

local_model = "llama3.1" # [Phi 3, Mistral, Gamma]
llm = ChatOllama(model=local_model, temperature = 0)

llm_json_mode = ChatOllama(model=local_model, temperature=0, format="json")

In [39]:
tools = [interactive_queue]
llm_with_tools = llm.bind_tools(tools)

In [40]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [41]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [35]:
lst = list(agent_executor.stream({"input": "Write an interactive queue submission on 1 debug node for 1 hour using HPCBot project"}))



> Entering new None chain...


ResponseError: <!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html><head>
<meta type="copyright" content="Copyright (C) 1996-2023 The Squid Software Foundation and contributors">
<meta http-equiv="Content-Type" CONTENT="text/html; charset=utf-8">
<title>ERROR: The requested URL could not be retrieved</title>
<style type="text/css"><!-- 
 /*
 * Copyright (C) 1996-2023 The Squid Software Foundation and contributors
 *
 * Squid software is distributed under GPLv2+ license and includes
 * contributions from numerous individuals and organizations.
 * Please see the COPYING and CONTRIBUTORS files for details.
 */

/*
 Stylesheet for Squid Error pages
 Adapted from design by Free CSS Templates
 http://www.freecsstemplates.org
 Released for free under a Creative Commons Attribution 2.5 License
*/

/* Page basics */
* {
	font-family: verdana, sans-serif;
}

html body {
	margin: 0;
	padding: 0;
	background: #efefef;
	font-size: 12px;
	color: #1e1e1e;
